In [1]:
gene_file_path = 'input/input_genes.txt'
phenotype_file_path = 'input/input_phenotype.txt'

In [2]:
import map_phenotype_to_gene
import collectVariantInfo
import pubmed
import ACMG
import filterVariantOnPhenotype
import csv

import pandas as pd
import numpy as np
import re
import myvariant
# from deepb.models import Main_table, Raw_input_table
from io import StringIO
from collections import Counter


# input_phenotype = 'data/sample_patient_phenotype.txt'
# input_genes = 'data/sample_genes.txt'

def format_hgvs(chrom, pos, ref, alt):
    '''get a valid hgvs name from VCF-style "chrom, pos, ref, alt" data.

    Example:

        >>> myvariant.format_hgvs("1", 35366, "C", "T")
        >>> myvariant.format_hgvs("2", 17142, "G", "GA")
        >>> myvariant.format_hgvs("MT", 8270, "CACCCCCTCT", "C")
        >>> myvariant.format_hgvs("X", 107930849, "GGA", "C")

    '''
    chrom = str(chrom)
    if chrom.lower().startswith('chr'):
        # trim off leading "chr" if any
        chrom = chrom[3:]
    if len(ref) == len(alt) == 1:
        # this is a SNP
        hgvs = 'chr{0}:g.{1}{2}>{3}'.format(chrom, pos, ref, alt)
    elif len(ref) > 1 and len(alt) == 1:
        # this is a deletion:
        if ref[0] == alt:
            start = int(pos) + 1
            end = int(pos) + len(ref) - 1
            hgvs = 'chr{0}:g.{1}_{2}del'.format(chrom, start, end)
        else:
            end = int(pos) + len(ref) - 1
            hgvs = 'chr{0}:g.{1}_{2}delins{3}'.format(chrom, pos, end, alt)
    elif len(ref) == 1 and len(alt) > 1:
        # this is a insertion
        if alt[0] == ref:
            hgvs = 'chr{0}:g.{1}_{2}ins'.format(chrom, pos, int(pos) + 1)
            ins_seq = alt[1:]
            hgvs += ins_seq
        else:
            hgvs = 'chr{0}:g.{1}delins{2}'.format(chrom, pos, alt)
    elif len(ref) > 1 and len(alt) > 1:
        end = int(pos) + len(alt) - 1
        hgvs = 'chr{0}:g.{1}_{2}delins{3}'.format(chrom, pos, end, alt)
    else:
        raise ValueError("Cannot convert {} into HGVS id.".format((chrom, pos, ref, alt)))
    return hgvs

def read_input_pheno_file(input_phenotype):
    if not input_phenotype:
        return '', ''
    text = StringIO(unicode(input_phenotype), newline=None)
    lines = text.readlines()
    lines = [line.strip() for line in lines]
    phenos = []
    for line in lines:
        if not line:
            continue
        phenos_each_line = re.split(r'  +|\t+|,|;|\.|\|', line.strip())
        phenos_each_line = [re.sub(r'^\W+|\W+$', '', s) for s in phenos_each_line]
        phenos_each_line = [s.lower() for s in phenos_each_line if s]
        phenos += phenos_each_line

    corner_cases = dict()
    for pheno in phenos:
        if re.search('development', pheno) and re.search('delay', pheno) and not re.search('growth', pheno):
            phenos.append('growth delay')
            corner_cases['growth delay'] = pheno.strip()
    for pheno in phenos:
        if re.search('growth', pheno) and re.search('delay', pheno) and not re.search('development', pheno):
            phenos.append('developmental delay')
            corner_cases['developmental delay'] = pheno.strip()
    return phenos, corner_cases

def read_input_gene_file(input_gene):
	candidate_vars = []
	input_gene = input_gene.split('\n')
	header = input_gene[0]
	sniffer = csv.Sniffer()
	dialect = sniffer.sniff(header)
	delimiter =  dialect.delimiter
	field_names = header.split(delimiter)
	chrom_idx, pos_idx, ref_idx, alt_idx, gene_idx = None, None, None, None, 0 

	for idx in xrange(len(field_names)):
		field = field_names[idx]
		if re.match(r'chrom', field, re.I): chrom_idx = idx
		if re.match(r'pos|start', field, re.I): pos_idx = idx
		if re.match(r'ref', field, re.I): ref_idx = idx
		if re.match(r'alt|allele 1', field, re.I): alt_idx = idx
		if re.match(r'gene (gene)|gene', field, re.I): gene_idx = idx

	input_gene_list = []
	CANDIDATE_GENES = []
	for line in input_gene[1:]:
		if not line:
			continue
		line = line.rstrip()
		parts = re.split(r'%s' % delimiter, line)
		input_gene_list.append(parts)
		gene = parts[gene_idx]
		CANDIDATE_GENES.append(gene)
		transcript, variant, variant_id = '', '', ''
		for part in parts:
			if re.search(r'_.*:c\.', part):
				transcript, variant = part.split(':')
			else:
				if re.search(r'c\.', part):
					variant = part
				if re.search(r'NM_', part, re.I):
					transcript = part.split(':')[0]
			if re.search(r'_.*:g\.', part):
				variant_id = 'chr' + part.split(':')[0].split('.')[-1] + part.split(':')[-1]
			if re.search(r'chr.*:g\.', part, re.I):
				variant_id = part
		if not variant_id and (chrom_idx and pos_idx and ref_idx and alt_idx):
			chrome, pos, ref, alt = parts[chrom_idx], parts[pos_idx], parts[ref_idx], parts[alt_idx]
			variant_id = format_hgvs(chrome, pos, ref, alt)
		candidate_vars.append((gene, variant, transcript, variant_id))

        # remove lines in the input file which has wrong number of fields
        field_nums = []
        for line in input_gene_list:
			field_nums.append(len(line))
        count = Counter(field_nums)
        correct_field_num = count.most_common()[0][0]
        correct_input_gene_list = []
        for line in input_gene_list:
			if len(line) == correct_field_num:
				correct_input_gene_list.append(line)
        df_genes = pd.DataFrame(correct_input_gene_list, columns = field_names)
        return candidate_vars, CANDIDATE_GENES, df_genes, field_names 

def map_phenotype2gene(CANDIDATE_GENES, phenos, corner_cases, candidate_vars):
	ranking_genes, ranking_disease = map_phenotype_to_gene.generate_score(phenos, CANDIDATE_GENES, corner_cases)
	# collect variant info
	hpo_filtered_genes = np.unique([i[0] for i in ranking_genes]).tolist()

	tmp_candidate_vars = []
	for var in candidate_vars:
		if var[0] in hpo_filtered_genes:
			tmp_candidate_vars.append(var)
	candidate_vars = tmp_candidate_vars
	return ranking_genes, candidate_vars

# 	status_step = "generating candidate variants ..." 
# 	raw_input = Raw_input_table.objects.get(id=raw_input_id)
# 	input_gene = raw_input.raw_input_gene
# 	input_phenotype = raw_input.raw_input_phenotype
input_gene = open("/Users/Tianqi/Desktop/xiaonantest1_g.txt",'rU').read()
input_phenotype = open("/Users/Tianqi/Desktop/xiaonantest1_p.txt",'rU').read()

# Read input pheno file and generate phenos and corner_cases 
phenos, corner_cases = read_input_pheno_file(input_phenotype)


# Read input gene file and generate candidate_vars. candidate_vars are (gene, variant, transcript, variant_id); CANDIDATE_GENES is a list of gene symbols; df_genes is a dataframe that keeps all the data that user uploaded; field_names are header of the input gene file 
candidate_vars, CANDIDATE_GENES, df_genes, field_names = read_input_gene_file(input_gene)

# map phenotype to gene; the candidate_vars was filtered: if it is a gene associated with phenos, then keep it.

if phenos:
#     raw_input.status = "Maping phenotypes to genes"
#     raw_input.save()
    ranking_genes, candidate_vars = map_phenotype2gene(CANDIDATE_GENES, phenos, corner_cases, candidate_vars)
else:
    ranking_genes = []
    for gene in CANDIDATE_GENES:
        ranking_genes.append((gene, 1.0, 1))

# collect variant info
# raw_input.status = "Annotating variants using genomic databases"
# raw_input.save()
mv = myvariant.MyVariantInfo()
final_res, variants = collectVariantInfo.get_variants(candidate_vars)

# pubmed
# raw_input.status = "Searching biomedical literatures"
# raw_input.save()
df_pubmed = pubmed.queryPubmedDB(final_res)

# ACMG
# raw_input.status = "Checking ACMG standard"
# raw_input.save()
df_hpo_ranking_genes = pd.DataFrame(ranking_genes, columns=['gene', 'score', 'hits'])
df_hpo_ranking_genes = df_hpo_ranking_genes[['gene', 'score']]
ACMG_result, variant_ACMG_interpretation, variant_ACMG_interpret_chinese = ACMG.Get_ACMG_result(df_hpo_ranking_genes, variants, df_pubmed)

# filter variant on phenotype

# if phenos:
# #     raw_input.status = "Filtering variants based on phenotypes"
# #     raw_input.save()
#     df_final_res, variant_ACMG_interpretation, variant_ACMG_interpret_chinese = filterVariantOnPhenotype.generateOutput(variants, ACMG_result, phenos, variant_ACMG_interpretation, variant_ACMG_interpret_chinese)
# else:
#     df_final_res = ACMG_result

ValueError: too many values to unpack

In [4]:
["<a href='https://www.ncbi.nlm.nih.gov/pubmed/%s'> %s </a>" %(i,i) for i in ['a','b']]


TypeError: not enough arguments for format string

In [11]:
ACMG_result

,gene,variant,id,final_score,pathogenicity_score,pathogenicity,hit_criteria,hpo_hit_score
0,BBS10,c.145C>T,chr12:g.76741994G>A,8.24,3.875000,Likely pathogenic,PS1|PM1|PP3|PP5,5.666667
1,HBB,c.208G>A,chr11:g.5247914C>T,8.11,2.625000,Uncertain significance,PM5|PM1|PP3|PP5,19.213095
2,DLX3,c.335A>G,chr17:g.48070945T>C,6.15,3.961200,Uncertain significance,PS1|PM1|PM2|PP3|BP1,2.000000
3,PROP1,c.652A>C,chr5:g.177419739T>G,5.23,2.250000,Uncertain significance,PM1|PM2|PP3,7.500000
4,SOX3,c.818C>T,chrX:g.139586408G>A,5.19,2.250000,Uncertain significance,PM1|PM2|PP3,7.333333
5,CEP152,c.3530A>G,chr15:g.49040744T>C,4.97,2.250000,Uncertain significance,PM1|PM2|PP3,6.404762
6,PRKDC,c.3278A>T,chr8:g.48813019T>A,4.45,2.250000,Uncertain significance,PM1|PM2|PP3,4.500000
7,PCDH15,c.4793G>A,chr10:g.55582714C>T,3.99,2.250000,Uncertain significance,PM1|PM2|PP3,3.166667
8,PLEC,c.1712G>A,chr8:g.145007482C>T,3.92,1.750000,Uncertain significance,PM1|PM2|PP3|BP1,6.666667
9,FOXRED1,c.286G>T,chr11:g.126141532G>T,3.47,1.500000,Uncertain significance,PM1|PP3,7.400000


In [29]:
gene_list = list(ACMG_result['gene'])
variant_list = list(ACMG_result['variant'])
gene_variant_list = [(gene_list[i], variant_list[i]) for i in range(len(gene_list))]

df_variant_ACMG_interpret = pd.DataFrame()
df_variant_ACMG_interpret_chinese = pd.DataFrame() 
for key in gene_variant_list:
    tmp_df = pd.DataFrame(variant_ACMG_interpretation[key], columns = ['criteria', 'interpretation'])
    tmp_df['gene'] = key[0]
    tmp_df['variant'] = key[1]
    tmp_df = tmp_df[['gene', 'variant', 'criteria', 'interpretation']] 
    df_variant_ACMG_interpret = pd.concat([df_variant_ACMG_interpret, tmp_df])

    tmp_df = pd.DataFrame(variant_ACMG_interpret_chinese[key], columns = ['criteria', 'interpretation'])
    tmp_df['gene'] = key[0]
    tmp_df['variant'] = key[1]
    tmp_df = tmp_df[['gene', 'variant', 'criteria', 'interpretation']] 
    df_variant_ACMG_interpret_chinese = pd.concat([df_variant_ACMG_interpret_chinese, tmp_df])

In [32]:
df_variant_ACMG_interpret_chinese

,gene,variant,criteria,interpretation
0,BBS10,c.145C>T,变异注释,"突变类型: missense_variant.<br/>蛋白功能区: GroEL-like equatorial domain.<br/>HGVS ID: chr12:g.76741994G>A.<br/>RefSeq ID: rs768933093.<br/>外显子: 1.<br/>ExAC 最小等位基因频率(MAF): 5.052e-05.<br/>DANN致病性分数: 0.999117068756.<br/>FATHMM致病性分数: 0.79186.<br/>MetaSVM致病性分数: 0.84564.<br/>GERP++序列保守性预测分数: 4.27.<br/>Clinvar数据库ID: 225010.<br/>Clinvar数据库记录的变异致病性: Pathogenic|Pathogenic.<br/>Clinvar数据库记录审核状态: criteria provided, single submitter.<br/>Clinvar数据库记录的Pubmed相关生物医学文献: ['25356970', '20120035', '16582908', '21044901', '20876674', '21642631', '20498079', '21517826', '20177705', '24746959', '25982971']."
1,BBS10,c.145C>T,PVS1,基因变异类型不是无效变异(null variant). 变异位点所在基因的功能丢失(loss of function)是已知的致病机制. 此变异不具有害的剪接效应(splicing effect). 不符合PVS1标准.
2,BBS10,c.145C>T,PS1和PM5,变异为错义突变. 之前报道导致相同氨基酸改变的基因变异被证明是致病的 (Clinvar数据库参考: 225010). 未发现导致相同氨基酸残基错义突变（不同氨基酸改变）导致的致病基因变异. 此变异不具有害的剪接效应(splicing effect). 符合PS1标准. 不符合PM5标准.
3,BBS10,c.145C>T,PS3和BS3,"[未发现针对此基因变异的完善的体内或体外功能性研究., 不符合PS3标准., 不符合BS3标准.]"
4,BBS10,c.145C>T,PS4,具有此基因变异的人群患病率并未显著升高，相对风险(Relative Risk)小于5.0.. 不符合PS4标准.
5,BBS10,c.145C>T,PM2,此基因变异的最小等位基因频率(MAF)极低(< 0.5%)，但并不引发隐性遗传病. 不符合PM2标准.
6,BBS10,c.145C>T,BA1和BS1,等位基因频率 <= 1%. 不符合BS1标准.
7,BBS10,c.145C>T,BS2,此基因变异在健康人中既不以隐性(纯合子)也不以显性(杂合子)状态存在. 不符合BS2标准.
8,BBS10,c.145C>T,PM1,此基因变异位于突变热点和/或关键的功能域(例如酶的活性部位)，且在这些区域不存在良性变异. 符合PM1标准.
9,BBS10,c.145C>T,PM4和BP3,变异位点在重复区域(repeat region). 不符合PM4标准. 不符合BP3标准.


In [31]:
df_variant_ACMG_interpret_chinese

,gene,variant,criteria,interpretation
0,BBS10,c.145C>T,变异注释,"突变类型: missense_variant.<br/>蛋白功能区: GroEL-like equatorial domain.<br/>HGVS ID: chr12:g.76741994G>A.<br/>RefSeq ID: rs768933093.<br/>外显子: 1.<br/>ExAC 最小等位基因频率(MAF): 5.052e-05.<br/>DANN致病性分数: 0.999117068756.<br/>FATHMM致病性分数: 0.79186.<br/>MetaSVM致病性分数: 0.84564.<br/>GERP++序列保守性预测分数: 4.27.<br/>Clinvar数据库ID: 225010.<br/>Clinvar数据库记录的变异致病性: Pathogenic|Pathogenic.<br/>Clinvar数据库记录审核状态: criteria provided, single submitter.<br/>Clinvar数据库记录的Pubmed相关生物医学文献: ['25356970', '20120035', '16582908', '21044901', '20876674', '21642631', '20498079', '21517826', '20177705', '24746959', '25982971']."
1,BBS10,c.145C>T,PVS1,基因变异类型不是无效变异(null variant). 变异位点所在基因的功能丢失(loss of function)是已知的致病机制. 此变异不具有害的剪接效应(splicing effect). 不符合PVS1标准.
2,BBS10,c.145C>T,PS1和PM5,变异为错义突变. 之前报道导致相同氨基酸改变的基因变异被证明是致病的 (Clinvar数据库参考: 225010). 未发现导致相同氨基酸残基错义突变（不同氨基酸改变）导致的致病基因变异. 此变异不具有害的剪接效应(splicing effect). 符合PS1标准. 不符合PM5标准.
3,BBS10,c.145C>T,PS3和BS3,"[未发现针对此基因变异的完善的体内或体外功能性研究., 不符合PS3标准., 不符合BS3标准.]"
4,BBS10,c.145C>T,PS4,具有此基因变异的人群患病率并未显著升高，相对风险(Relative Risk)小于5.0.. 不符合PS4标准.
5,BBS10,c.145C>T,PM2,此基因变异的最小等位基因频率(MAF)极低(< 0.5%)，但并不引发隐性遗传病. 不符合PM2标准.
6,BBS10,c.145C>T,BA1和BS1,等位基因频率 <= 1%. 不符合BS1标准.
7,BBS10,c.145C>T,BS2,此基因变异在健康人中既不以隐性(纯合子)也不以显性(杂合子)状态存在. 不符合BS2标准.
8,BBS10,c.145C>T,PM1,此基因变异位于突变热点和/或关键的功能域(例如酶的活性部位)，且在这些区域不存在良性变异. 符合PM1标准.
9,BBS10,c.145C>T,PM4和BP3,变异位点在重复区域(repeat region). 不符合PM4标准. 不符合BP3标准.


In [7]:
variant_ACMG_interpretation[('BBS10', 'c.145C>T')]

[('variant_annotations',
  u"Effect: missense_variant.<br/>Protein domain: GroEL-like equatorial domain.<br/>HGVS ID: chr12:g.76741994G>A.<br/>RefSeq ID: rs768933093.<br/>exon: 1.<br/>ExAC MAF: 5.052e-05.<br/>DANN pathogenicity score: 0.999117068756.<br/>FATHMM pathogenicity score: 0.79186.<br/>MetaSVM pathogenicity score: 0.84564.<br/>GERP++ conservation score: 4.27.<br/>Clinvar variation ids: 225010.<br/>Pathogenicity reported by Clinvar: Pathogenic|Pathogenic.<br/>Clinvar review status: criteria provided, single submitter.<br/>Pubmed references from Clinvar: ['25356970', '20120035', '16582908', '21044901', '20876674', '21642631', '20498079', '21517826', '20177705', '24746959', '25982971']."),
 ('PVS1',
  'Variant effect NOT in null variant type. Allele in a gene where loss of function (LOF) is a known mechanism of disease. The variant does NOT have damaging splicing effect. PVS1 is NOT met.'),
 ('PS1 and PM5',
  'Variant effect is missense. Same amino acid change as a previously est

In [10]:
df_final_res.head()

,gene,variant,protein,id,final_score,pathogenicity,hit_criteria,pathogenicity_score,hpo_hit_score,pheno_match_score
76,BBS10,c.145C>T,p.Arg49Trp,chr12:g.76741994G>A,9.511010,Likely pathogenic,PS1|PM1|PP3|PP5,3.8750,5.666667,1.154249
103,HBB,c.208G>A,p.Gly70Ser,chr11:g.5247914C>T,8.713391,Uncertain significance,PM5|PM1|PP3|PP5,2.6250,19.213095,1.074401
66,DLX3,c.335A>G,p.Lys112Arg,chr17:g.48070945T>C,6.150000,Uncertain significance,PS1|PM1|PM2|PP3|BP1,3.9612,2.000000,1.000000
53,PROP1,c.652A>C,p.Ser218Arg,chr5:g.177419739T>G,5.230000,Uncertain significance,PM1|PM2|PP3,2.2500,7.500000,1.000000
107,SOX3,c.818C>T,p.Ser273Leu,chrX:g.139586408G>A,5.190000,Uncertain significance,PM1|PM2|PP3,2.2500,7.333333,1.000000


In [11]:
df_genes.head()

,Chromosome,Start,Stop,Reference,Allele 1,Allele 2,Allele In Scope,Transcript,Gene Profile Report,Gene (gene),cDNA (cNomen),Protein (pNomen),HGVS cDNA-level nomenclature (fullCNomen)
0,1,1635536,1635536,.,.,A,A,NM_024011.2,,CDK11A,c.1735dupT,p.Y579Lfs*66,NM_024011.2:c.1735dupT
1,1,23713843,23713843,T,C,T,C,NM_003196.2,,TCEA3,c.889A>G,p.M297V,NM_003196.2:c.889A>G
2,1,24882663,24882663,A,A,G,G,NM_001010980.4,,NCMAP,c.-8+3A>G,,NM_001010980.4:c.-8+3A>G
3,1,26663842,26663842,G,G,A,A,NM_001039775.3,,AIM1L,c.3673C>T,p.R1225W,NM_001039775.3:c.3673C>T
4,1,39896387,39896387,A,C,A,C,NM_012090.5,,MACF1,c.10958A>C,p.E3653A,NM_012090.5:c.10958A>C


In [8]:
phenos

[u'bilateral post-axial polydactyly of hands and feet',
 u'macrocephaly',
 u'tall stature',
 u'post-natal',
 u'central hypotonia',
 u'global developmental delay',
 u'obstructive sleep apnea',
 u'delayed cns central nervous myelination',
 u'white matter',
 u'corpus callosum',
 u'mildly ectopic neurohypophys posterior pituitary upslanted palpebral fissures',
 u'epicanthus',
 u'arched eyebrows',
 u'craniofacial asymmetry',
 u'bronchial stenosis',
 u'macrosomia',
 u'recurrent infection',
 u'gastroesophageal reflux',
 u'supratentorial perivascular space',
 u'mucopolysaccharidosis plasma urine creatine',
 u'purine',
 u'pyrimidine metabolism',
 u'gross motor',
 u'language',
 u'cognitive',
 u'ventriculomegaly',
 u'metatarsal bones',
 u'proximal phalanges',
 u'obesity',
 u'wheeze',
 u'brachycephaly frontal prominence',
 u'bitemporal narrowing',
 u'clinodactyly',
 u'brachydactyly',
 u'bardet-biedl',
 'growth delay',
 'developmental delay']

In [9]:
field_names

['Chromosome',
 'Start',
 'Stop',
 'Reference',
 'Allele 1',
 'Allele 2',
 'Allele In Scope',
 'Transcript',
 'Gene Profile Report',
 'Gene (gene)',
 'cDNA (cNomen)',
 'Protein (pNomen)',
 'HGVS cDNA-level nomenclature (fullCNomen)']

In [17]:
variant_ACMG_interpretation.head()

,gene,variant,criteria,interpretation
0,BBS10,c.145C>T,variant_annotations,"Effect: missense_variant.<br/>Protein domain: GroEL-like equatorial domain.<br/>HGVS ID: chr12:g.76741994G>A.<br/>RefSeq ID: rs768933093.<br/>exon: 1.<br/>ExAC MAF: 5.052e-05.<br/>DANN pathogenicity score: 0.999117068756.<br/>FATHMM pathogenicity score: 0.79186.<br/>MetaSVM pathogenicity score: 0.84564.<br/>GERP++ conservation score: 4.27.<br/>Clinvar variation ids: 225010.<br/>Pathogenicity reported by Clinvar: Pathogenic|Pathogenic.<br/>Clinvar review status: criteria provided, single submitter.<br/>Pubmed references from Clinvar: ['25356970', '20120035', '16582908', '21044901', '20876674', '21642631', '20498079', '21517826', '20177705', '24746959', '25982971']."
1,BBS10,c.145C>T,PVS1,Variant effect NOT in null variant type. Allele in a gene where loss of function (LOF) is a known mechanism of disease. The variant does NOT have damaging splicing effect. PVS1 is NOT met.
2,BBS10,c.145C>T,PS1 and PM5,Variant effect is missense. Same amino acid change as a previously established pathogenic variant regardless of nucleotide change (Clinvar references: 225010). Not find missense change at an amino acid residue where a different missense change determined to be pathogenic has been seen before. The variant does NOT have damaging splicing effect. PS1 is met. PM5 is NOT met.
3,BBS10,c.145C>T,PS3 and BS3,Not find well-established functional studies on this variant. PS3 is NOT met. BS3 is NOT met.
4,BBS10,c.145C>T,PS4,Relative risk is smaller than 5. The prevalence of the variant in affected individuals is NOT significantly increased compared with the prevalence in controls. PS4 is NOT met.


In [18]:
variant_ACMG_interpret_chinese.head()

,基因,变异,标准,解读
0,BBS10,c.145C>T,变异注释,"突变类型: missense_variant.<br/>蛋白功能区: GroEL-like equatorial domain.<br/>HGVS ID: chr12:g.76741994G>A.<br/>RefSeq ID: rs768933093.<br/>外显子: 1.<br/>ExAC 最小等位基因频率(MAF): 5.052e-05.<br/>DANN致病性分数: 0.999117068756.<br/>FATHMM致病性分数: 0.79186.<br/>MetaSVM致病性分数: 0.84564.<br/>GERP++序列保守性预测分数: 4.27.<br/>Clinvar数据库ID: 225010.<br/>Clinvar数据库记录的变异致病性: 致病|致病.<br/>Clinvar数据库记录审核状态: 提供标准，单个提交者.<br/>Clinvar数据库记录的Pubmed相关生物医学文献: ['25356970', '20120035', '16582908', '21044901', '20876674', '21642631', '20498079', '21517826', '20177705', '24746959', '25982971']."
1,BBS10,c.145C>T,PVS1,基因变异类型不是无效变异(null variant). 变异位点所在基因的功能丢失(loss of function)是已知的致病机制. 此变异不具有害的剪接效应(splicing effect). 不符合PVS1标准.
2,BBS10,c.145C>T,PS1和PM5,变异为错义突变. 之前报道导致相同氨基酸改变的基因变异被证明是致病的 (Clinvar数据库参考: 225010). 未发现导致相同氨基酸残基错义突变（不同氨基酸改变）导致的致病基因变异. 此变异不具有害的剪接效应(splicing effect). 符合PS1标准. 不符合PM5标准.
3,BBS10,c.145C>T,PS3和BS3,未发现针对此基因变异的完善的体内或体外功能性研究. 不符合PS3标准. 不符合BS3标准.
4,BBS10,c.145C>T,PS4,具有此基因变异的人群患病率并未显著升高，相对风险(Relative Risk)小于5.0.. 不符合PS4标准.


In [ ]:
df_final_res, df_genes, phenos, field_names, variant_ACMG_interpretation, variant_ACMG_interpret_chinese

In [14]:
nput_gene = df_genes.to_json(orient='records')
input_phenotype = ', '.join(phenos)
result_table = ACMG_result.to_json(orient='records')
interpretation = variant_ACMG_interpretation.to_json(orient='records')
interpretation_chinese = variant_ACMG_interpret_chinese.to_json(orient='records')
# logger.info("Finish processing data, start writing data to DB in background main task")


# sample = Main_table(
#     task_id=raw_input_id,
#     input_gene=input_gene,
#     input_phenotype=input_phenotype,
#     result=result_table,
#     interpretation=interpretation,
#     interpretation_chinese=interpretation_chinese,
#     pub_date=timezone.now(),
#     user_name=raw_input.user_name,
#     task_name=raw_input.task_name,
# )
# sample.save()
# logger.info("Finish writing data to DB in background main task")
# raw_input.status = "succeed"
# raw_input.save()

In [16]:
interpretation[:1000]

'[{"gene":"BBS10","variant":"c.145C>T","criteria":"variant_annotations","interpretation":"Effect: missense_variant.<br\\/>Protein domain: GroEL-like equatorial domain.<br\\/>HGVS ID: chr12:g.76741994G>A.<br\\/>RefSeq ID: rs768933093.<br\\/>exon: 1.<br\\/>ExAC MAF: 5.052e-05.<br\\/>DANN pathogenicity score: 0.999117068756.<br\\/>FATHMM pathogenicity score: 0.79186.<br\\/>MetaSVM pathogenicity score: 0.84564.<br\\/>GERP++ conservation score: 4.27.<br\\/>Clinvar variation ids: 225010.<br\\/>Pathogenicity reported by Clinvar: Pathogenic|Pathogenic.<br\\/>Clinvar review status: criteria provided, single submitter.<br\\/>Pubmed references from Clinvar: [\'25356970\', \'20120035\', \'16582908\', \'21044901\', \'20876674\', \'21642631\', \'20498079\', \'21517826\', \'20177705\', \'24746959\', \'25982971\']."},{"gene":"BBS10","variant":"c.145C>T","criteria":"PVS1","interpretation":"Variant effect NOT in null variant type. Allele in a gene where loss of function (LOF) is a known mechanism of dis

In [21]:
interpretation_chinese[:1000]

'[{"\\u57fa\\u56e0":"BBS10","\\u53d8\\u5f02":"c.145C>T","\\u6807\\u51c6":"\\u53d8\\u5f02\\u6ce8\\u91ca","\\u89e3\\u8bfb":"\\u7a81\\u53d8\\u7c7b\\u578b: missense_variant.<br\\/>\\u86cb\\u767d\\u529f\\u80fd\\u533a: GroEL-like equatorial domain.<br\\/>HGVS ID: chr12:g.76741994G>A.<br\\/>RefSeq ID: rs768933093.<br\\/>\\u5916\\u663e\\u5b50: 1.<br\\/>ExAC \\u6700\\u5c0f\\u7b49\\u4f4d\\u57fa\\u56e0\\u9891\\u7387(MAF): 5.052e-05.<br\\/>DANN\\u81f4\\u75c5\\u6027\\u5206\\u6570: 0.999117068756.<br\\/>FATHMM\\u81f4\\u75c5\\u6027\\u5206\\u6570: 0.79186.<br\\/>MetaSVM\\u81f4\\u75c5\\u6027\\u5206\\u6570: 0.84564.<br\\/>GERP++\\u5e8f\\u5217\\u4fdd\\u5b88\\u6027\\u9884\\u6d4b\\u5206\\u6570: 4.27.<br\\/>Clinvar\\u6570\\u636e\\u5e93ID: 225010.<br\\/>Clinvar\\u6570\\u636e\\u5e93\\u8bb0\\u5f55\\u7684\\u53d8\\u5f02\\u81f4\\u75c5\\u6027: \\u81f4\\u75c5|\\u81f4\\u75c5.<br\\/>Clinvar\\u6570\\u636e\\u5e93\\u8bb0\\u5f55\\u5ba1\\u6838\\u72b6\\u6001: \\u63d0\\u4f9b\\u6807\\u51c6\\uff0c\\u5355\\u4e2a\\u63d0\\u4ea4\

In [ ]:
interpretation_chinese = variant_ACMG_interpret_chinese.to_json(orient='records')

In [35]:
round(2.7)

3.0

In [36]:
8/3

2

In [8]:
aa = ''

In [9]:
["<a href='https://www.ncbi.nlm.nih.gov/clinvar/variation/%s/'> %s </a>" %(i,i) for i in aa.split('|')]

["<a href='https://www.ncbi.nlm.nih.gov/clinvar/variation//'>  </a>"]

In [19]:
if bb:
    b=1+1

In [20]:
b

NameError: name 'b' is not defined

In [17]:
bb = []

In [18]:
["<a href='https://www.ncbi.nlm.nih.gov/pubmed/%s'> %s </a>" %(i,i) for i in bb]

[]

In [10]:
import csv
import pandas as pd
import numpy as np
import re
import myvariant
from io import StringIO
from collections import Counter

In [2]:
input_gene = open("/Users/Tianqi/Desktop/vcfsample.txt",'rU').read()

In [3]:
candidate_vars = []
input_gene = input_gene.split('\n')

for line in input_gene:
    if line and line[:2] != "##":
        header = line
        sniffer = csv.Sniffer()
        dialect = sniffer.sniff(header)
        delimiter =  dialect.delimiter
        field_names = header.split(delimiter)
        break
        
chrom_idx, pos_idx, ref_idx, alt_idx, gene_idx = None, None, None, None, None

for idx in xrange(len(field_names)):
    field = field_names[idx]
    if re.match(r'chrom|#chrom', field, re.I): chrom_idx = idx
    if re.match(r'pos|start', field, re.I): pos_idx = idx
    if re.match(r'ref', field, re.I): ref_idx = idx
    if re.match(r'alt|allele 1', field, re.I): alt_idx = idx
    if re.match(r'gene (gene)|gene', field, re.I): gene_idx = idx

input_gene_list = []
CANDIDATE_GENES = []
for line in input_gene[1:]:
    if not line:
        continue
    line = line.rstrip()
    parts = re.split(r'%s' % delimiter, line)
    if line.startswith('#'):
        continue
    input_gene_list.append(parts)
    gene, transcript, variant, variant_id = '', '', '', ''
    if gene_idx:
        gene = parts[gene_idx]
        CANDIDATE_GENES.append(gene)
    for part in parts:
        if re.search(r'_.*:c\.', part):
            transcript, variant = part.split(':')
        else:
            if re.search(r'c\.', part):
                variant = part
            if re.search(r'NM_', part, re.I):
                transcript = part.split(':')[0]
        if re.search(r'_.*:g\.', part):
            variant_id = 'chr' + part.split(':')[0].split('.')[-1] + part.split(':')[-1]
        if re.search(r'chr.*:g\.', part, re.I):
            variant_id = part
    if not variant_id and (chrom_idx is not None and pos_idx is not None and ref_idx is not None and alt_idx is not None):
        chrome, pos, ref, alt = parts[chrom_idx], parts[pos_idx], parts[ref_idx], parts[alt_idx]
        variant_id = format_hgvs(chrome, pos, ref, alt)
    candidate_vars.append((gene, variant, transcript, variant_id))

# remove lines in the input file which has wrong number of fields
field_nums = []
for line in input_gene_list:
    field_nums.append(len(line))
count = Counter(field_nums)
correct_field_num = count.most_common()[0][0]
correct_input_gene_list = []
for line in input_gene_list:
    if len(line) == correct_field_num:
        correct_input_gene_list.append(line)
df_genes = pd.DataFrame(correct_input_gene_list, columns = field_names)

In [4]:
df_genes

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NA00001,NA00002,NA00003
0,20,14370,rs6054257,G,A,29,PASS,NS=3;DP=14;AF=0.5;DB;H2,GT:GQ:DP:HQ,"0|0:48:1:51,51","1|0:48:8:51,51","1/1:43:5:.,."


In [5]:
candidate_vars

[('', '', '', 'chr20:g.14370G>A')]

In [1]:
import map_phenotype_to_gene
import collectVariantInfo
import pubmed
import ACMG
import filterVariantOnPhenotype
import csv

import pandas as pd
import numpy as np
import re
import myvariant
# from deepb.models import Main_table, Raw_input_table
from io import StringIO
from collections import Counter


# input_phenotype = 'data/sample_patient_phenotype.txt'
# input_genes = 'data/sample_genes.txt'

def format_hgvs(chrom, pos, ref, alt):
    '''get a valid hgvs name from VCF-style "chrom, pos, ref, alt" data.

    Example:

        >>> myvariant.format_hgvs("1", 35366, "C", "T")
        >>> myvariant.format_hgvs("2", 17142, "G", "GA")
        >>> myvariant.format_hgvs("MT", 8270, "CACCCCCTCT", "C")
        >>> myvariant.format_hgvs("X", 107930849, "GGA", "C")

    '''
    chrom = str(chrom)
    if chrom.lower().startswith('chr'):
        # trim off leading "chr" if any
        chrom = chrom[3:]
    if len(ref) == len(alt) == 1:
        # this is a SNP
        hgvs = 'chr{0}:g.{1}{2}>{3}'.format(chrom, pos, ref, alt)
    elif len(ref) > 1 and len(alt) == 1:
        # this is a deletion:
        if ref[0] == alt:
            start = int(pos) + 1
            end = int(pos) + len(ref) - 1
            hgvs = 'chr{0}:g.{1}_{2}del'.format(chrom, start, end)
        else:
            end = int(pos) + len(ref) - 1
            hgvs = 'chr{0}:g.{1}_{2}delins{3}'.format(chrom, pos, end, alt)
    elif len(ref) == 1 and len(alt) > 1:
        # this is a insertion
        if alt[0] == ref:
            hgvs = 'chr{0}:g.{1}_{2}ins'.format(chrom, pos, int(pos) + 1)
            ins_seq = alt[1:]
            hgvs += ins_seq
        else:
            hgvs = 'chr{0}:g.{1}delins{2}'.format(chrom, pos, alt)
    elif len(ref) > 1 and len(alt) > 1:
        end = int(pos) + len(alt) - 1
        hgvs = 'chr{0}:g.{1}_{2}delins{3}'.format(chrom, pos, end, alt)
    else:
        raise ValueError("Cannot convert {} into HGVS id.".format((chrom, pos, ref, alt)))
    return hgvs

def read_input_pheno_file(input_phenotype):
    if not input_phenotype:
        return '', ''
    text = StringIO(unicode(input_phenotype), newline=None)
    lines = text.readlines()
    lines = [line.strip() for line in lines]
    phenos = []
    for line in lines:
        if not line:
            continue
        phenos_each_line = re.split(r'  +|\t+|,|;|\.|\|', line.strip())
        phenos_each_line = [re.sub(r'^\W+|\W+$', '', s) for s in phenos_each_line]
        phenos_each_line = [s.lower() for s in phenos_each_line if s]
        phenos += phenos_each_line

    corner_cases = dict()
    for pheno in phenos:
        if re.search('development', pheno) and re.search('delay', pheno) and not re.search('growth', pheno):
            phenos.append('growth delay')
            corner_cases['growth delay'] = pheno.strip()
    for pheno in phenos:
        if re.search('growth', pheno) and re.search('delay', pheno) and not re.search('development', pheno):
            phenos.append('developmental delay')
            corner_cases['developmental delay'] = pheno.strip()
    return phenos, corner_cases

def read_input_gene_file(input_gene):
    candidate_vars = []
    input_gene = input_gene.split('\n')

    for line in input_gene:
        if line and line[:2] != "##":
            header = line
            sniffer = csv.Sniffer()
            dialect = sniffer.sniff(header)
            delimiter =  dialect.delimiter
            field_names = header.split(delimiter)
            break

    chrom_idx, pos_idx, ref_idx, alt_idx, gene_idx = None, None, None, None, None

    for idx in xrange(len(field_names)):
        field = field_names[idx]
        if re.match(r'chrom|#chrom', field, re.I): chrom_idx = idx
        if re.match(r'pos|start', field, re.I): pos_idx = idx
        if re.match(r'ref', field, re.I): ref_idx = idx
        if re.match(r'alt|allele 1', field, re.I): alt_idx = idx
        if re.match(r'gene (gene)|gene', field, re.I): gene_idx = idx

    input_gene_list = []
    CANDIDATE_GENES = []
    for line in input_gene[1:]:
        if not line:
            continue
        line = line.rstrip()
        parts = re.split(r'%s' % delimiter, line)
        if line.startswith('#'):
            continue
        input_gene_list.append(parts)
        gene, transcript, variant, variant_id = '', '', '', ''
        if gene_idx:
            gene = parts[gene_idx]
            CANDIDATE_GENES.append(gene)
        for part in parts:
            if re.search(r'_.*:c\.', part):
                transcript, variant = part.split(':')
            else:
                if re.search(r'c\.', part):
                    variant = part
                if re.search(r'NM_', part, re.I):
                    transcript = part.split(':')[0]
            if re.search(r'_.*:g\.', part):
                variant_id = 'chr' + part.split(':')[0].split('.')[-1] + part.split(':')[-1]
            if re.search(r'chr.*:g\.', part, re.I):
                variant_id = part
        if not variant_id and (chrom_idx is not None and pos_idx is not None and ref_idx is not None and alt_idx is not None):
            chrome, pos, ref, alt = parts[chrom_idx], parts[pos_idx], parts[ref_idx], parts[alt_idx]
            variant_id = format_hgvs(chrome, pos, ref, alt)
        candidate_vars.append((gene, variant, transcript, variant_id))

    # remove lines in the input file which has wrong number of fields
    field_nums = []
    for line in input_gene_list:
        field_nums.append(len(line))
    count = Counter(field_nums)
    correct_field_num = count.most_common()[0][0]
    correct_input_gene_list = []
    for line in input_gene_list:
        if len(line) == correct_field_num:
            correct_input_gene_list.append(line)
    df_genes = pd.DataFrame(correct_input_gene_list, columns = field_names)
    return candidate_vars, CANDIDATE_GENES, df_genes, field_names 

def map_phenotype2gene(CANDIDATE_GENES, phenos, corner_cases, candidate_vars):
	ranking_genes, ranking_disease = map_phenotype_to_gene.generate_score(phenos, CANDIDATE_GENES, corner_cases)
	# collect variant info
	hpo_filtered_genes = np.unique([i[0] for i in ranking_genes]).tolist()

	tmp_candidate_vars = []
	for var in candidate_vars:
		if var[0] in hpo_filtered_genes:
			tmp_candidate_vars.append(var)
	candidate_vars = tmp_candidate_vars
	return ranking_genes, candidate_vars

# def master_function(raw_input_id):
# 	status_step = "generating candidate variants ..." 
# 	raw_input = Raw_input_table.objects.get(id=raw_input_id)
# 	input_gene = raw_input.raw_input_gene
# 	input_phenotype = raw_input.raw_input_phenotype
input_gene = open("/Users/Tianqi/Desktop/vcfsample.txt",'rU').read()
input_phenotype = ''
# Read input pheno file and generate phenos and corner_cases 
phenos, corner_cases = read_input_pheno_file(input_phenotype)


# Read input gene file and generate candidate_vars. candidate_vars are (gene, variant, transcript, variant_id); CANDIDATE_GENES is a list of gene symbols; df_genes is a dataframe that keeps all the data that user uploaded; field_names are header of the input gene file 
candidate_vars, CANDIDATE_GENES, df_genes, field_names = read_input_gene_file(input_gene)

if not CANDIDATE_GENES:
    # collect variant info
#     raw_input.status = "Annotating variants using genomic databases"
#     raw_input.save()
    final_res, variants = collectVariantInfo.get_variants_from_vcf(candidate_vars)
    CANDIDATE_GENES = [_[0] for _ in final_res]
    # map phenotype to gene; the candidate_vars was filtered: if it is a gene associated with phenos, then keep it.
    if phenos:
#         raw_input.status = "Maping phenotypes to genes"
#         raw_input.save()
        ranking_genes, candidate_vars = map_phenotype2gene(CANDIDATE_GENES, phenos, corner_cases, candidate_vars)
    else:
        ranking_genes = []
        for gene in CANDIDATE_GENES:
            ranking_genes.append((gene, 1.0, 1))
else:
    # map phenotype to gene; the candidate_vars was filtered: if it is a gene associated with phenos, then keep it.
    if phenos:
#         raw_input.status = "Maping phenotypes to genes"
#         raw_input.save()
        ranking_genes, candidate_vars = map_phenotype2gene(CANDIDATE_GENES, phenos, corner_cases, candidate_vars)
    else:
        ranking_genes = []
        for gene in CANDIDATE_GENES:
            ranking_genes.append((gene, 1.0, 1))
    # collect variant info
#     raw_input.status = "Annotating variants using genomic databases"
#     raw_input.save()
    final_res, variants = collectVariantInfo.get_variants(candidate_vars)

if final_res == [] and variants == defaultdict(dict):
#     return None, df_genes, phenos, field_names, None, None
    1
else:
    # pubmed
    # raw_input.status = "Searching biomedical literatures"
    # raw_input.save()
    df_pubmed = pubmed.queryPubmedDB(final_res)

    # ACMG
    # raw_input.status = "Checking ACMG standard"
    # raw_input.save()
    df_hpo_ranking_genes = pd.DataFrame(ranking_genes, columns=['gene', 'score', 'hits'])
    df_hpo_ranking_genes = df_hpo_ranking_genes[['gene', 'score']]
    ACMG_result, variant_ACMG_interpretation, variant_ACMG_interpret_chinese, df_variant_ACMG_interpret, df_variant_ACMG_interpret_chinese = ACMG.Get_ACMG_result(df_hpo_ranking_genes, variants, df_pubmed)

    # filter variant on phenotype

    if phenos:
    #     raw_input.status = "Filtering variants based on phenotypes"
    #     raw_input.save()
        df_final_res, variant_ACMG_interpretation, variant_ACMG_interpret_chinese = filterVariantOnPhenotype.generateOutput(variants, ACMG_result, phenos, variant_ACMG_interpretation, variant_ACMG_interpret_chinese)
    #     return df_final_res, df_genes, phenos, field_names, variant_ACMG_interpretation, variant_ACMG_interpret_chinese

    # else:
    #     return ACMG_result, df_genes, phenos, field_names, df_variant_ACMG_interpret, df_variant_ACMG_interpret_chinese

TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
final_res, variants = collectVariantInfo.get_variants_from_vcf(candidate_vars)

In [23]:
candidate_vars

[('', '', '', 'chr20:g.14370G>A')]

In [2]:
final_res

[(u'PTPN11', u'c.923A>G', u'p.Asn308Ser')]

In [21]:
from collections import defaultdict

In [4]:
variants

defaultdict(dict,
            {(u'PTPN11', u'c.923A>G'): {'alt': u'G',
              'clinvar_pathogenicity': u'Pathogenic|Pathogenic|Pathogenic|Pathogenic',
              'clinvar_pmids': [],
              'clinvar_review_status': '',
              'clinvar_variation_ids': '',
              'dann': '0.998400682813',
              'dbscSNV_ada_score': '',
              'dbscSNV_rf_score': '',
              'effect': u'missense_variant',
              'exon': u'8',
              'fathmm': '0.99101',
              'gene': u'PTPN11',
              'gerp++': '5.64',
              'id': u'chr12:g.112915524A>G',
              'interpro_domain': [u'PTP type protein phosphatase|Protein-tyrosine phosphatase-like'],
              'maf_1000g': '',
              'maf_esp6500': '',
              'maf_exac': '',
              'metasvm': '0.99878',
              'protein': u'p.Asn308Ser',
              'ref': u'A',
              'rsid': u'rs121918455',
              'transcript': u'NM_002834.3',
    

In [11]:
pubmed.queryPubmedDB(final_res)

KeyError: ('CD59', 'c.146delA')

In [7]:
df_genes

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NA00001,NA00002,NA00003
0,12,112915524,rs6054257,A,G,29,PASS,NS=3;DP=14;AF=0.5;DB;H2,GT:GQ:DP:HQ,"0|0:48:1:51,51","1|0:48:8:51,51","1/1:43:5:.,."


In [4]:
a = '[{"#CHROM":"16","POS":"50745926","ID":"rs2066844","REF":"C","ALT":"T","QUAL":"80","FILTER":"PASS","INFO":"NS=3;DP=14;AF=0.5;DB;H2","FORMAT":"GT:GQ:DP:HQ","NA00001":"0|0:48:1:51,51","NA00002":"1|0:48:8:51,51","NA00003":"1\/1:43:5:.,."},{"#CHROM":"20","POS":"14370","ID":"rs6054257","REF":"G","ALT":"A","QUAL":"29","FILTER":"PASS","INFO":"NS=3;DP=14;AF=0.5;DB;H2","FORMAT":"GT:GQ:DP:HQ","NA00001":"0|0:48:1:51,51","NA00002":"1|0:48:8:51,51","NA00003":"1\/1:43:5:.,."},{"#CHROM":"20","POS":"17330","ID":".","REF":"T","ALT":"A","QUAL":"3","FILTER":"q10","INFO":"NS=3;DP=11;AF=0.017","FORMAT":"GT:GQ:DP:HQ","NA00001":"0|0:49:3:58,50","NA00002":"0|1:3:5:65,3","NA00003":"0\/0:41:3"},{"#CHROM":"20","POS":"1110696","ID":"rs6040355","REF":"A","ALT":"G,T","QUAL":"67","FILTER":"PASS","INFO":"NS=2;DP=10;AF=0.333,0.667;AA=T;DB","FORMAT":"GT:GQ:DP:HQ","NA00001":"1|2:21:6:23,27","NA00002":"2|1:2:0:18,2","NA00003":"2\/2:35:4"},{"#CHROM":"20","POS":"1230237","ID":".","REF":"T","ALT":"G","QUAL":"47","FILTER":"PASS","INFO":"NS=3;DP=13;AA=T","FORMAT":"GT:GQ:DP:HQ","NA00001":"0|0:54:7:56,60","NA00002":"0|0:48:4:51,51","NA00003":"0\/0:61:2"},{"#CHROM":"20","POS":"1230288","ID":".","REF":"T","ALT":".","QUAL":"50","FILTER":"PASS","INFO":"NS=3;DP=13;AA=T","FORMAT":"GT:GQ:DP:HQ","NA00001":"0|0:54:7:56,60","NA00002":"0|0:48:4:51,51","NA00003":"0\/0:61:2"},{"#CHROM":"20","POS":"1234567","ID":"microsat1","REF":"GTCT","ALT":"G,GTACT","QUAL":"50","FILTER":"PASS","INFO":"NS=3;DP=9;AA=G","FORMAT":"GT:GQ:DP","NA00001":"0\/1:35:4","NA00002":"0\/2:17:2","NA00003":"1\/1:40:3"}]'

In [7]:
a.split("},{")

['[{"#CHROM":"16","POS":"50745926","ID":"rs2066844","REF":"C","ALT":"T","QUAL":"80","FILTER":"PASS","INFO":"NS=3;DP=14;AF=0.5;DB;H2","FORMAT":"GT:GQ:DP:HQ","NA00001":"0|0:48:1:51,51","NA00002":"1|0:48:8:51,51","NA00003":"1\\/1:43:5:.,."',
 '"#CHROM":"20","POS":"14370","ID":"rs6054257","REF":"G","ALT":"A","QUAL":"29","FILTER":"PASS","INFO":"NS=3;DP=14;AF=0.5;DB;H2","FORMAT":"GT:GQ:DP:HQ","NA00001":"0|0:48:1:51,51","NA00002":"1|0:48:8:51,51","NA00003":"1\\/1:43:5:.,."',
 '"#CHROM":"20","POS":"17330","ID":".","REF":"T","ALT":"A","QUAL":"3","FILTER":"q10","INFO":"NS=3;DP=11;AF=0.017","FORMAT":"GT:GQ:DP:HQ","NA00001":"0|0:49:3:58,50","NA00002":"0|1:3:5:65,3","NA00003":"0\\/0:41:3"',
 '"#CHROM":"20","POS":"1110696","ID":"rs6040355","REF":"A","ALT":"G,T","QUAL":"67","FILTER":"PASS","INFO":"NS=2;DP=10;AF=0.333,0.667;AA=T;DB","FORMAT":"GT:GQ:DP:HQ","NA00001":"1|2:21:6:23,27","NA00002":"2|1:2:0:18,2","NA00003":"2\\/2:35:4"',
 '"#CHROM":"20","POS":"1230237","ID":".","REF":"T","ALT":"G","QUAL":"47

In [50]:
b='[{"Gene":"AARS","Transcript":"NM_001605.2","cDNA (cNomen)":"c.1786-5T>C","Protein (pNomen)":"","Target Position":"chr16:70293094-70293094"},{"Gene":"AP4E1","Transcript":"NM_007347.4","cDNA (cNomen)":"c.2553A>T","Protein (pNomen)":"p.Glu851Asp","Target Position":"chr15:51289729-51289729"},{"Gene":"ARFGEF2","Transcript":"NM_006420.2","cDNA (cNomen)":"c.3120C>T","Protein (pNomen)":"p.Leu1040Leu","Target Position":"chr20:47611134-47611134"},{"Gene":"ARL2BP","Transcript":"NM_012106.3","cDNA (cNomen)":"c.207C>T","Protein (pNomen)":"p.Tyr69Tyr","Target Position":"chr16:57282555-57282555"},{"Gene":"ASPM","Transcript":"NM_018136.4","cDNA (cNomen)":"c.7812A>T","Protein (pNomen)":"p.Lys2604Asn","Target Position":"chr1:197070569-197070569"},{"Gene":"CD96","Transcript":"NM_198196.2","cDNA (cNomen)":"c.1295A>T","Protein (pNomen)":"p.Gln432Leu","Target Position":"chr3:111342667-111342667"},{"Gene":"CHAMP1","Transcript":"NM_001164144.1","cDNA (cNomen)":"c.449_451delCTC","Protein (pNomen)":"p.Pro150del","Target Position":"chr13:115089762-115089766"},{"Gene":"CREBBP","Transcript":"NM_004380.2","cDNA (cNomen)":"c.5354G>A","Protein (pNomen)":"p.Cys1785Tyr","Target Position":"chr16:3779694-3779694"},{"Gene":"DYNC1H1","Transcript":"NM_001376.4","cDNA (cNomen)":"c.634A>G","Protein (pNomen)":"p.Met212Val","Target Position":"chr14:102446171-102446171"},{"Gene":"DYNC2H1","Transcript":"NM_001080463.1","cDNA (cNomen)":"c.2479A>G","Protein (pNomen)":"p.Ile827Val","Target Position":"chr11:103006582-103006582"},{"Gene":"ERCC6L2","Transcript":"NM_020207.4","cDNA (cNomen)":"c.4123A>C","Protein (pNomen)":"p.Thr1375Pro","Target Position":"chr9:98774922-98774922"},{"Gene":"FRAS1","Transcript":"NM_025074.6","cDNA (cNomen)":"c.3065A>C","Protein (pNomen)":"p.Lys1022Thr","Target Position":"chr4:79295319-79295319"},{"Gene":"GBA2","Transcript":"NM_020944.2","cDNA (cNomen)":"c.379C>T","Protein (pNomen)":"p.Arg127Trp","Target Position":"chr9:35744684-35744684"},{"Gene":"KAT6B","Transcript":"NM_012330.3","cDNA (cNomen)":"c.4097_4105dupAAGAGGAAG","Protein (pNomen)":"p.Glu1366_Glu1368dup","Target Position":"chr10:76788659-76788660"},{"Gene":"KLLN","Transcript":"NM_001126049.1","cDNA (cNomen)":"c.17C>T","Protein (pNomen)":"p.Pro6Leu","Target Position":"chr10:89622228-89622228"},{"Gene":"LARP7","Transcript":"NM_016648.3","cDNA (cNomen)":"c.-2-4A>G","Protein (pNomen)":"","Target Position":"chr4:113565820-113565820"},{"Gene":"LIFR","Transcript":"NM_002310.5","cDNA (cNomen)":"c.46G>A","Protein (pNomen)":"p.Asp16Asn","Target Position":"chr5:38530704-38530704"},{"Gene":"MC1R","Transcript":"NM_002386.3","cDNA (cNomen)":"c.892T>C","Protein (pNomen)":"p.Tyr298His","Target Position":"chr16:89986558-89986558"},{"Gene":"MCM4","Transcript":"NM_005914.3","cDNA (cNomen)":"c.2102G>A","Protein (pNomen)":"p.Arg701Gln","Target Position":"chr8:48885590-48885590"},{"Gene":"NBAS","Transcript":"NM_015909.3","cDNA (cNomen)":"c.5110C>T","Protein (pNomen)":"p.His1704Tyr","Target Position":"chr2:15427225-15427225"},{"Gene":"NBAS","Transcript":"NM_015909.3","cDNA (cNomen)":"c.2290C>T","Protein (pNomen)":"p.Pro764Ser","Target Position":"chr2:15601374-15601374"},{"Gene":"NFKB1","Transcript":"NM_003998.3","cDNA (cNomen)":"c.1601G>A","Protein (pNomen)":"p.Arg534His","Target Position":"chr4:103518782-103518782"},{"Gene":"PAH","Transcript":"NM_000277.1","cDNA (cNomen)":"c.1208C>T","Protein (pNomen)":"p.Ala403Val","Target Position":"chr12:103234285-103234285"},{"Gene":"PDE6C","Transcript":"NM_006204.3","cDNA (cNomen)":"c.2087C>T","Protein (pNomen)":"p.Thr696Met","Target Position":"chr10:95418708-95418708"},{"Gene":"PIGN","Transcript":"NM_176787.4","cDNA (cNomen)":"c.1372G>A","Protein (pNomen)":"p.Ala458Thr","Target Position":"chr18:59780429-59780429"},{"Gene":"POC1B","Transcript":"NM_172240.2","cDNA (cNomen)":"c.179A>G","Protein (pNomen)":"p.Lys60Arg","Target Position":"chr12:89891041-89891041"},{"Gene":"PRF1","Transcript":"NM_001083116.1","cDNA (cNomen)":"c.1528T>C","Protein (pNomen)":"p.Cys510Arg","Target Position":"chr10:72357949-72357949"},{"Gene":"PROP1","Transcript":"NM_006261.4","cDNA (cNomen)":"c.652A>C","Protein (pNomen)":"p.Ser218Arg","Target Position":"chr5:177419739-177419739"},{"Gene":"RELN","Transcript":"NM_005045.3","cDNA (cNomen)":"c.8795C>A","Protein (pNomen)":"p.Ser2932Tyr","Target Position":"chr7:103138572-103138572"},{"Gene":"RYR1","Transcript":"NM_000540.2","cDNA (cNomen)":"c.8959G>C","Protein (pNomen)":"p.Glu2987Gln","Target Position":"chr19:39001164-39001164"},{"Gene":"SH3PXD2B","Transcript":"NM_001017995.2","cDNA (cNomen)":"c.921G>C","Protein (pNomen)":"p.Gln307His","Target Position":"chr5:171777458-171777458"},{"Gene":"SLC39A4","Transcript":"NM_130849.3","cDNA (cNomen)":"c.1126G>A","Protein (pNomen)":"p.Ala376Thr","Target Position":"chr8:145639669-145639669"},{"Gene":"SLC4A4","Transcript":"NM_003759.3","cDNA (cNomen)":"c.2542T>C","Protein (pNomen)":"p.Phe848Leu","Target Position":"chr4:72413417-72413417"},{"Gene":"SMPD1","Transcript":"NM_000543.4","cDNA (cNomen)":"c.108_109insGCG","Protein (pNomen)":"p.Val36_Leu37insAla","Target Position":"chr11:6411935-6411936"},{"Gene":"SRCAP","Transcript":"NM_006662.2","cDNA (cNomen)":"c.7853A>G","Protein (pNomen)":"p.Asn2618Ser","Target Position":"chr16:30749214-30749214"},{"Gene":"TCOF1","Transcript":"NM_001135243.1","cDNA (cNomen)":"c.925G>C","Protein (pNomen)":"p.Gly309Arg","Target Position":"chr5:149753791-149753791"},{"Gene":"TG","Transcript":"NM_003235.4","cDNA (cNomen)":"c.2222C>T","Protein (pNomen)":"p.Thr741Met","Target Position":"chr8:133900274-133900274"},{"Gene":"TMPRSS15","Transcript":"NM_002772.2","cDNA (cNomen)":"c.687T>G","Protein (pNomen)":"p.Phe229Leu","Target Position":"chr21:19737543-19737543"},{"Gene":"UBR1","Transcript":"NM_174916.2","cDNA (cNomen)":"c.3959T>C","Protein (pNomen)":"p.Leu1320Pro","Target Position":"chr15:43281055-43281055"},{"Gene":"ZNF513","Transcript":"NM_144631.5","cDNA (cNomen)":"c.901G>A","Protein (pNomen)":"p.Gly301Arg","Target Position":"chr2:27601137-27601137"}]'

In [51]:
[i.split(":")[0][1:-1] for i in b.split("},{")[0][2:].split(',')]

['Gene', 'Transcript', 'cDNA (cNomen)', 'Protein (pNomen)', 'Target Position']

In [52]:
[i.split('":"')[0][1:-1] for i in a.split("},{")[0][2:].split('","')]

['#CHRO',
 'O',
 '',
 'E',
 'L',
 'UA',
 'ILTE',
 'NF',
 'ORMA',
 'A0000',
 'A0000',
 'A0000']

In [38]:
input_gene_field

['#CHRO',
 'O',
 '',
 'E',
 'L',
 'UA',
 'ILTE',
 'NF',
 'ORMA',
 'A0000',
 'A0000',
 'A0000']

In [94]:
[i.split(":")[0][1:-1] for i in a.split("},{")[0][2:].split(',')]

['#CHROM',
 'POS',
 'ID',
 'REF',
 'ALT',
 'QUAL',
 'FILTER',
 'INFO',
 'FORMAT',
 'NA00001',
 '1',
 'NA00002',
 '1',
 'NA00003',
 '']

In [93]:
[i.split('":"')[0] for i in a.split("},{")[0][3:].split('","')]

['#CHROM',
 'POS',
 'ID',
 'REF',
 'ALT',
 'QUAL',
 'FILTER',
 'INFO',
 'FORMAT',
 'NA00001',
 'NA00002',
 'NA00003']

In [88]:
b.split("},{")[0][3:]

'Gene":"AARS","Transcript":"NM_001605.2","cDNA (cNomen)":"c.1786-5T>C","Protein (pNomen)":"","Target Position":"chr16:70293094-70293094"'

In [4]:
from langdetect import detect

In [7]:
detect(u"任务1")

'zh-cn'

In [20]:
from oauth2client.service_account import ServiceAccountCredentials

# scopes = ['https://www.googleapis.com/auth/translateservice.admin']

credentials = ServiceAccountCredentials.from_json_keyfile_name(
    '/Users/Tianqi/Keep_Learning/google_api/translate-7696ad4e40b3.json')

In [21]:
from httplib2 import Http

http_auth = credentials.authorize(Http())

In [22]:
from apiclient.discovery import build
service = build('translate', 'v2', http=http_auth)

Traceback (most recent call last):
  File "/Users/Tianqi/anaconda/lib/python2.7/site-packages/googleapiclient/discovery_cache/__init__.py", line 41, in autodetect
    from . import file_cache
  File "/Users/Tianqi/anaconda/lib/python2.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    'file_cache is unavailable when using oauth2client >= 4.0.0')
ImportError: file_cache is unavailable when using oauth2client >= 4.0.0


TypeError: cannot instantiate ctype 'EVP_MD_CTX' of unknown size

In [ ]:
service = build('translate', 'v2',
        developerKey='e37b6a3f5406868430b2b58d7b65825e4cb21082')
service.translations().list(
  source='en',
  target='fr',
  q=['flower', 'car']
).execute()

In [64]:
# Imports the Google Cloud client library
from google.cloud import translate

# Instantiates a client
translate_client = translate.Client()

# The text to translate
text = u'Hello, world!'
# The target language
target = 'ru'

# Translates some text into Russian
translation = translate_client.translate(
    text,
    target_language=target)

print(u'Text: {}'.format(text))
print(u'Translation: {}'.format(translation['translatedText']))

Forbidden: 403 Daily Limit Exceeded (GET https://translation.googleapis.com/language/translate/v2?target=ru&q=Hello%2C+world%21)

In [13]:
from google.cloud import translate

translate_client = translate.Client()
text = u'Hello'
target = 'zh'

translation = translate_client.translate(text,target_language=target)
print(u'Translation: {}'.format(translation['translatedText']))

Translation: 你好


In [15]:
text = u'为什么时而好用时而不好用'
target = 'en'
translation = translate_client.translate(text,target_language=target)
translation['translatedText']

u'Why sometimes easy to use and sometimes easy to use'

In [24]:
sourceLang = 'zh-Hans'
targetLang = 'en'
sourceText = '你好啊'

url = "https://translate.googleapis.com/translate_a/single?client=gtx&sl=" + sourceLang + "&tl=" + targetLang + "&dt=t&q=" + sourceText

In [22]:
url

'https://translate.googleapis.com/translate_a/single?client=gtx&sl=zh-Hans&tl=en&dt=t&q=\xe4\xbd\xa0\xe5\xa5\xbd\xe5\x95\x8a'

In [35]:
url = 'response'

In [ ]:
https://translate.googleapis.com/translate_a/single?client=gtx&sl=zh-Hans&tl=en&dt=t&q=%E4%BD%A0%E5%A5%BD

In [31]:
import urllib

f = urllib.urlopen(url)
myfile = f.read()
print myfile

<!DOCTYPE html><html lang=en><meta charset=utf-8><meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width"><title>Error 403 (Forbidden)!!1</title><style>*{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) no-repeat 0% 0%/100% 100%;-moz-border-image:url(//www.google.com/images/

In [37]:
import urllib, json
response = urllib.urlopen(url)
data = json.loads(response.read())
print data

ValueError: No JSON object could be decoded

In [139]:
iput_text = "不发育，严重的全球发育迟缓，身材矮小，高血压，小头畸形，皮层视觉障碍，趾骨平滑肌，重叠趾，球鼻，鼻腔不发达，延迟牙喷，额叶多毛症，巩膜葡萄膜炎，垂直性低张力，颞视盘，松散的食管括约肌，俱乐部脚，畸形特征，营养不良，先天性CMV，失神经注视，下旋转海马，胼，体，血管瘤，胎记，额头多毛症，眼睛，听觉"

In [115]:
iput_text = "不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育，不发育"

In [136]:
iput_text = "不发育"

In [140]:
import urllib2
site = "https://translate.googleapis.com/translate_a/single?client=gtx&sl=zh-Hans&tl=en&dt=t&q="+iput_text
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
        'Accept-Encoding': 'none',
        'Accept-Language': 'en-US,en;q=0.8',
        'Connection': 'keep-alive'}
req = urllib2.Request(site, headers=hdr)
try:
    page = urllib2.urlopen(req)
except urllib2.HTTPError, e:
    print e

content = page.read()
translate = json.loads(content)[0][0][0]
print translate

Non-developed, severe global growth retardation, short stature, high blood pressure, small head deformity, cortical visual impairment, phalanx smooth muscle, overlapping toes, ball nose, nasal inactivity, delayed toothbrush, frontal lobelia, scleral uveitis, 


In [141]:
json.loads(content)

[[[u'Non-developed, severe global growth retardation, short stature, high blood pressure, small head deformity, cortical visual impairment, phalanx smooth muscle, overlapping toes, ball nose, nasal inactivity, delayed toothbrush, frontal lobelia, scleral uveitis, ',
   u'\u4e0d\u53d1\u80b2\uff0c\u4e25\u91cd\u7684\u5168\u7403\u53d1\u80b2\u8fdf\u7f13\uff0c\u8eab\u6750\u77ee\u5c0f\uff0c\u9ad8\u8840\u538b\uff0c\u5c0f\u5934\u7578\u5f62\uff0c\u76ae\u5c42\u89c6\u89c9\u969c\u788d\uff0c\u8dbe\u9aa8\u5e73\u6ed1\u808c\uff0c\u91cd\u53e0\u8dbe\uff0c\u7403\u9f3b\uff0c\u9f3b\u8154\u4e0d\u53d1\u8fbe\uff0c\u5ef6\u8fdf\u7259\u55b7\uff0c\u989d\u53f6\u591a\u6bdb\u75c7\uff0c\u5de9\u819c\u8461\u8404\u819c\u708e\uff0c',
   None,
   None,
   3],
  [u'Vertical, low tension, temporal disc, loose esophageal sphincter, club foot, malformations, malnutrition, congenital CMV, denervated gyrus, lower hippocampus, corpuscles, body, hemangioma, birthmark, forehead hirsutism, eye, hearing',
   u'\u5782\u76f4\u6027\u4f4

In [147]:
import map_phenotype_to_gene
import collectVariantInfo
import pubmed
import ACMG
import filterVariantOnPhenotype
import csv

import pandas as pd
import numpy as np
import re
import myvariant
# from deepb.models import Main_table, Raw_input_table
from io import StringIO
from collections import Counter
from collections import defaultdict
from langdetect import detect
import urllib2
import json


In [148]:
def read_input_pheno_file(input_phenotype):
	if not input_phenotype:
		return '', '', ''
	language = detect(unicode(input_phenotype))
	if language == "zh-cn":
		site = "https://translate.googleapis.com/translate_a/single?client=gtx&sl=zh-Hans&tl=en&dt=t&q="+input_phenotype
		hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
				'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
				'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
				'Accept-Encoding': 'none',
				'Accept-Language': 'en-US,en;q=0.8',
				'Connection': 'keep-alive'}
		req = urllib2.Request(site, headers=hdr)
		try:
			page = urllib2.urlopen(req)
			content = page.read()
			input_phenotype = json.loads(content)[0][0][0]
		except urllib2.HTTPError, e:
			return '', '', ''
	text = StringIO(unicode(input_phenotype), newline=None)
	lines = text.readlines()
	lines = [line.strip() for line in lines]
	phenos = []
	for line in lines:
		if not line:
			continue
        phenos_each_line = re.split(r'  +|\t+|,|;|\.|\|', line.strip())
        phenos_each_line = [re.sub(r'^\W+|\W+$', '', s) for s in phenos_each_line]
        phenos_each_line = [s.lower() for s in phenos_each_line if s]
        phenos += phenos_each_line

	corner_cases = dict()
	original_phenos = [_.strip() for _ in phenos]
	for pheno in phenos:
		if re.search('development', pheno) and re.search('delay', pheno) and not re.search('growth', pheno):
			phenos.append('growth delay')
			orner_cases['growth delay'] = pheno.strip()
		for pheno in phenos:
			if re.search('growth', pheno) and re.search('delay', pheno) and not re.search('development', pheno):
				phenos.append('developmental delay')
				corner_cases['developmental delay'] = pheno.strip()
		phenos = [_.strip() for _ in phenos]
		phenos = list(set(phenos))
		return phenos, corner_cases, original_phenos

In [149]:
read_input_pheno_file('Lamdoid, Sagittal, Craniosynostosis, Metopic, macular atrophy, anetoderma, short stature, developmental delay, Ventriculomegaly, growth hormone deficiency')

([u'short stature',
  u'craniosynostosis',
  u'ventriculomegaly',
  u'metopic',
  u'growth hormone deficiency',
  u'anetoderma',
  u'lamdoid',
  u'macular atrophy',
  u'developmental delay',
  u'sagittal'],
 {},
 [u'lamdoid',
  u'sagittal',
  u'craniosynostosis',
  u'metopic',
  u'macular atrophy',
  u'anetoderma',
  u'short stature',
  u'developmental delay',
  u'ventriculomegaly',
  u'growth hormone deficiency'])

In [20]:
import pandas as pd
import re
from nltk.corpus import wordnet as wn
from collections import deque
import re
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter
import os.path
import numpy as np
BASE = '/Users/Tianqi/Keep_Learning/bio-nlp/deepbrain/deepb/'

"""
For each input phenotype of the patient, we do preliminary mapping of it to hpo standard terms.
We convert the input phenotype term and hpo standard terms to a word list and strip symbols in the word and lemmatize the word.
The word list of hpo standard term and its synonyms and the word list of input phenotype are compared and if they are a match or partial match and they do not contain non-overlapped words with opposite meanings, we consider the hpo term as a candidate.
The matched hpo terms are filtered on their common ancestors, which basically reflect semantics of the hpo terms. The hpo term with the highest similarity score is used as the gold standard. If another hpo candidate has a very high level common ancestor (not close in semantics) with it, then that hpo candidate will be removed. Matches after this filtering step are direct matches (from input phenotype).
We also get indirect matches by mapping the synonyms of hpo term with very high similarity score to other hpo terms. 
The direct and indirect matches are combined and if they do not have non-overlapped opposite meaning words, they are taken into the final matches.

The hpo dictionary is like a graph. It has a root node.
hpo_subclass: dictionary with a hpo term (id) as key and its direct children as value.
hpo_superclass: dictionary with a hpo term (id) as key and its direct parents as value.
"""

HPO_SUBCLASS = os.path.join(BASE, "data/hpo_subclasses.txt")
HPO_SUPERCLASSES = os.path.join(BASE, "data/hpo_superclasses.txt")
HPO_NAMES = os.path.join(BASE, "data/hpo_names.txt")
HPO_FILE = os.path.join(BASE, "data/all_hpo_terms_and_synonyms.txt")
PHENOTYPE_TO_GENE_FILE = os.path.join(BASE, "data/Expanded_OMIM_ALL_FREQUENCIES_phenotype_to_genes_without_synonym.txt")
PHENOTYPE_TO_DISEASE_FILE = os.path.join(BASE, 'data/Expanded_ALL_SOURCES_ALL_FREQUENCIES_diseases_to_phenotypes.txt')

## Get levels of each hpo term; e.g., HP:0000001 is the first level; its direct children are the second level.
## Phenotypic abnormality is level 1; Abnormality of prenatal development or birth is level 2.
## In the search for the common ancestors, we should stop at level 2; if two terms have the common ancestor below or equal at level 2, then we deem them to be semantically similar.

PHENOTYPE_COUNT_FILE = os.path.join(BASE, 'data/hpoid_count_in_pubmed.txt')
df_pheno_count = pd.read_csv(PHENOTYPE_COUNT_FILE, sep = '\t', usecols = [0, 2])
hpoidpheno_count = df_pheno_count.set_index('hpoid')['count'].to_dict()

hpo_subclass = dict()
with open(HPO_SUBCLASS, 'rb') as f:
    # skip the first header line
    f.readline()
    for line in f.readlines():
        line = line.rstrip()
        parts = line.split('\t')
        hpoid = parts[0]
        subclasses = parts[1].strip('[]').replace("'", "")
        subclasses = [_.strip() for _ in subclasses.split(',')]
        hpo_subclass[hpoid] = subclasses

hpo_levels = dict()
level = 0
root = 'HP:0000001'
curr_level_nodes = [root] 
while curr_level_nodes:
    next_level_nodes = []
    for node in curr_level_nodes:
        hpo_levels[node] = level
        if node in hpo_subclass:
            next_level_nodes += hpo_subclass[node]
    curr_level_nodes = next_level_nodes
    level += 1

hpo_superclass = dict()
with open(HPO_SUPERCLASSES, 'rb') as f:
    # skip the first header line
    f.readline()
    for line in f.readlines():
        line = line.rstrip()
        parts = line.split('\t')
        hpoid = parts[0]
        superclasses = parts[1].strip('[]').replace("'", "")
        superclasses = [_.strip() for _ in superclasses.split(',')]
        hpo_superclass[hpoid] = superclasses

hpo_name = dict()
with open(HPO_NAMES, 'rb') as f:
    # Skip the first header line
    f.readline()
    for line in f.readlines():
        line = line.rstrip()
        parts = line.split('\t')
        hpoid = parts[0]
        hponame = parts[1]
        hpo_name[hpoid] = hponame        

def getAllAncestorsBFS(node, tree):
    """ get all ancestors of a node using BFS

    Args:
        node (string): node in a tree
        tree (dict): dictionary with node as key and its direct parents as value

    Returns:
        P (list): the ancestors of a node with the first element as the direct parent and so on

    """
    P, Q = [node], deque([node])
    while Q:
        u = Q.popleft()
        if u not in tree:
            continue
        for v in tree[u]:
            if v in P: continue
            if v not in tree: continue
            P.append(v)
            Q.append(v)
    # Append All ('HP:0000001') as the highest ancestor 
    P.append('HP:0000001')
    return P 

def findLowestCommonAncestor(node1, node2, tree):
    """ find the lowest common ancestor of two nodes in a tree

    Args:
        node1 (string): hpoid of a phenotype
        node2 (string): hpoid of another phenotype
        tree (dict): dictionary with key as a node and value as the direct parents of the node

    Returns:
        lca (tuple): lowest common ancestor (hpoid, level, hponame)
        common_ancestors (list): common ancestors from lowest common ancestor to top level common ancestor

    """
    node1_ancestors = getAllAncestorsBFS(node1, tree) 
    node2_ancestors = getAllAncestorsBFS(node2, tree)
    common_ancestors = [] 
    for ancestor in node2_ancestors:
        if ancestor in node1_ancestors:
            name = hpo_name[ancestor]
            level = hpo_levels[ancestor]
            common_ancestors.append((ancestor, level, name)) 
    lca = common_ancestors[0]
    return lca, common_ancestors

def get_antonyms(word):
    """ generate antonyms of a word using NLTK and wordnet

    Args:
        word (string): input word

    Returns:
         antonyms (set): a set of antonyms of the input word

    """
    antonyms = []
    for synset in wn.synsets(word):
        for lemma in synset.lemmas():
            antonyms += lemma.antonyms()
            similar_synsets = synset.similar_tos() 
            for similar_synset in similar_synsets:
                for lemma_ in similar_synset.lemmas():
                    antonyms += lemma_.antonyms()
    direct_antonyms = list(set(antonyms))
    # Get similar words of the direct antonyms
    for antonym in direct_antonyms:
        similar_synsets = antonym.synset().similar_tos()
        for synset in similar_synsets:
            for lemma in synset.lemmas():
                antonyms.append(lemma)
    antonyms = list(set(antonyms))
    antonyms = set([_.name() for _ in antonyms])
    return antonyms

with open(HPO_FILE, 'rb') as infile:
    hpo_name2id = dict()
    hpo_id2synonyms = dict()
    # skip the first line
    infile.readline()
    for line in infile.readlines():
        line = line.rstrip()
        parts = line.split('\t')
        hpoid, hponame = parts[0], parts[1]
        hpo_name2id[hponame] = hpoid
        hpoid = hpoid.split('-')[0]
        if hponame in hpo_id2synonyms:
            hpo_id2synonyms[hpoid].append(hponame)
        else:
            hpo_id2synonyms[hpoid] = [hponame]
    hpo_terms = hpo_name2id.keys()  

processed_antonyms = dict()
def isOppositeMeaning(words_in_pheno_only, words_in_hpo_only):
    """ check if the input two word lists contain words with opposite meaning

    Args:
        words_in_pheno_only (set): a set of words that appear in the phenotype word list only
        word_in_hpo_only (set): a set of words that appear in the hpo term word list only

    Returns:
         bool: True or False

    """
    global processed_antonyms
    # If any word ends with ness, convert to its noun form
    words_in_pheno_only |= set([_[0:-4] for _ in words_in_pheno_only if _.endswith('ness')])
    words_in_hpo_only |= set([_[0:-4] for _ in words_in_hpo_only if _.endswith('ness')])
    # corner case: gain
    if 'gain' in words_in_hpo_only:
        words_in_hpo_only.add('increased')
        words_in_hpo_only.add('steady')
    if 'gain' in words_in_pheno_only:
        words_in_pheno_only.add('increased')
        words_in_pheno_only.add('steady')
    # Poor weight gain and weight gain are in opposite meaning
    if (words_in_pheno_only == set(['poor']) and words_in_hpo_only == set()) or (words_in_hpo_only == set(['poor']) and words_in_pheno_only == set()):
        return True
    for word in words_in_pheno_only:
        try:
            if word in processed_antonyms:
                antonyms = processed_antonyms[word]
            else:
                antonyms = get_antonyms(word)
                processed_antonyms[word] = antonyms
        except UnicodeDecodeError as e:
            continue
        if antonyms.intersection(words_in_hpo_only):
            return True
    for word in words_in_hpo_only:
        try:
            if word in processed_antonyms:
                antonyms = processed_antonyms[word]
            else:
                antonyms = get_antonyms(word)
                processed_antonyms[word] = antonyms
        except UnicodeDecodeError as e:
            continue
        if antonyms.intersection(words_in_pheno_only):
            return True
    return False 

def processhpoterms(hpo_):
    """ process each hpo term by transforming each term to a list of individual words, and strip ,()' in the word
        and lemmatize each word. Use a global dictionary to record the mapping from original hpo term to the processed list.
        Use re for split hpo terms
        Use lemmatizer to lemmatize words e.g., disturbances --> distrubance

    Args:
        hpo_ (string); a hpo term

    Returns:
        None

    """
    global hpo_words_dict
    hpo_words =  [_.strip(",()'") for _ in re.split(' |/', hpo_)]
    try:
        hpo_words = [lemmatizer.lemmatize(word) for word in hpo_words]
    except UnicodeDecodeError as e:
        pass
    hpo_words_dict[hpo_] = hpo_words

# If the opposite meanings are found for a hpoid, then its synonyms are all in opposite meanings
unmatches_due_to_opposite_meanings = []
hpo_words_dict = dict()
# Use lemmatizer to lemmatize words e.g., disturbances --> distrubance
lemmatizer = WordNetLemmatizer()

def map2hpo(pheno):
    """ preliminary mapping of each input phenotype of the patient to hpo standard terms

    Args:
        pheno (string): an input phenotype of the patient

    Returns:
        matches (list): each element is a tuple (hpoid, hponame, match_type, similarity_score)
            e.g., [('HP:0012171', 'Stereotypical hand wringing', 'OVERLAP', 0.6666666666666666), ...]
    """
    global unmatches_due_to_opposite_meanings, hpo_words_dict
    matches = []
    # Use lemmatizer to lemmatize words e.g., disturbances --> distrubance
    pheno_ = pheno.lower()
    pheno_words = [_.strip(",()'") for _ in re.split(' |/', pheno_)]
    try:
        pheno_words = [lemmatizer.lemmatize(word) for word in pheno_words]
    except UnicodeDecodeError as e:
        pass

    for hpo in hpo_terms:
        hpoid = hpo_name2id[hpo]
        hpo_ = hpo.lower()
        if hpo_.startswith('obsolete '):
            continue 
        if pheno_ == hpo_:
            matches.append((hpoid, hpo, 'EXACT', 1.0)) 
            continue
        if hpo_ not in hpo_words_dict:
            processhpoterms(hpo_)
        hpo_words = hpo_words_dict[hpo_]

        if pheno_words == hpo_words:
            matches.append((hpoid, hpo, 'LEMMA_MATCH', 1.0)) 
            continue
        if set(pheno_words) == set(hpo_words):
            matches.append((hpoid, hpo, 'REORDER', 1.0)) 
            continue

        common_words = set(pheno_words) & set(hpo_words)
        if not common_words:
            continue

        words_in_pheno_only = set(pheno_words) - common_words
        words_in_hpo_only = set(hpo_words) - common_words
        # check if the pheno and hpo are in opposite meaning
        # if any word in pheno is an antonym of a word in hpo, or the other way
        # we skip this hpo
        sim = float(len(common_words)) / max(len(hpo_words), len(pheno_words))  
        if (sim >= 0.3333) and (pheno_ in hpo_) and (len(hpo_words) <= 10 * len(pheno_words)):
            if not isOppositeMeaning(words_in_pheno_only, words_in_hpo_only):
                matches.append((hpoid, hpo, 'SUB', sim))
            else:
                unmatches_due_to_opposite_meanings.append(hpoid.split('-')[0])
            continue
        if (sim >= 0.3333) and (hpo_ in pheno_) and (len(pheno_words) <= 10 * len(hpo_words)): 
            if not isOppositeMeaning(words_in_pheno_only, words_in_hpo_only):
                matches.append((hpoid, hpo, 'SUPER', sim))
            else:
                unmatches_due_to_opposite_meanings.append(hpoid.split('-')[0])
            continue
        if (sim >= 0.5) and len(pheno_words) >= 2:
            if not isOppositeMeaning(words_in_pheno_only, words_in_hpo_only):
                matches.append((hpoid, hpo, 'OVERLAP', sim))
            else:
                unmatches_due_to_opposite_meanings.append(hpoid.split('-')[0])
    matches_res = []
    for match in matches:
        if match[0].split('-')[0] not in unmatches_due_to_opposite_meanings:
            matches_res.append(match)
    matches = matches_res
    return matches

def filterMatchesOnCommonAncestors(matches):
    """ this function filters matches on common ancestors. One input phenotype may map to multiple hpo standard terms
        with different similarity score. We pick up the one with best match score, and check the common ancestor of
        other hpo terms with it. The common ancestor is literally in the cengji jigou, e.g., abnormality of head -- abnormality
        of eye -- baineichang. If the common ancestor of a hpo term and the best-match hpo term are very high level, then they are likely NOT
        describing the same phenotype. This hpo term will be removed from the matches.

    Args:
        matches (list): each element is a tuple (hpoid, hponame, match_type, similarity_score)
            e.g., [('HP:0012171', 'Stereotypical hand wringing', 'OVERLAP', 0.6666666666666666), ...]

    Returns:
        final_matches (list): each element is a tuple (hponame, hpoid, similarity_score).
            e.g., [('Postnatal macrocephaly','HP:0005490',0.5), ('Macrocephaly,relative','HP:0004482-synonym', 0.5), ...]

    """
    if not matches:
        return matches 
    final_matches = []
    # If the match is a disease not a phenotype, then we just add it to the final matches result
    matches_tmp = []
    for match in matches:
        if not match[0].startswith('HP:'):
            final_matches.append((match[1], match[0], match[3]))
        else:
            matches_tmp.append(match)
    matches = matches_tmp
    if not matches:
        return final_matches
    best_match_hpo = sorted(matches, key = lambda x: x[3], reverse = True)[0][0]
    for match in matches:
        term = match[0]
        # Remove 'synonym' from the hpoid (term) e.g., HP:0004414-synonym --> HP:0004414
        term = term.split('-')[0]
        lca, common_ancestors = findLowestCommonAncestor(term, best_match_hpo.split('-')[0], hpo_superclass)
        if lca[1] >= 2:
            final_matches.append((match[1], match[0], match[3])) 
    return final_matches    

def map2hpoWithPhenoSynonyms(pheno):
    """ map each input phenotype of the patient to hpo standard terms

    Args:
        pheno (string): an input phenotype of the patient 

    Returns: 
        final_matches (list): each element is a tuple (hponame, hpoid, similarity_score)
            e.g., [('Postnatal macrocephaly','HP:0005490',0.5), ('Macrocephaly,relative','HP:0004482-synonym', 0.5), ...]

    """
    global unmatches_due_to_opposite_meanings
    matches = map2hpo(pheno)
    direct_matches = filterMatchesOnCommonAncestors(matches)  
    final_matches = direct_matches 
    for match in direct_matches:
        hponame = match[0]
        hpoid = match[1].split('-')[0]
        sim = match[2]
        if sim >= 0.75:
            hposynonyms = hpo_id2synonyms[hpoid]
            for synonym in hposynonyms:
                if synonym != hponame:
                    matches_ = map2hpo(synonym)
                    indirect_matches = filterMatchesOnCommonAncestors(matches_)
                    final_matches = final_matches + indirect_matches
    final_matches = list(set(final_matches))
    matches_res = []
    for match in final_matches:
        if match[0].split('-')[0] not in unmatches_due_to_opposite_meanings:
            matches_res.append(match)
    final_matches = matches_res
    unmatches_due_to_opposite_meanings = []
    return final_matches

hpoid2gene = dict()
with open(PHENOTYPE_TO_GENE_FILE, 'rb') as f:
    # skip the first header line
    f.readline()
    for line in f.readlines():
        line = line.rstrip()
        parts = line.split('\t')
        geneid, genesymbol, hponame, hpoid = parts[0], parts[1], parts[2], parts[3]
        if hpoid in hpoid2gene:
            hpoid2gene[hpoid].append(genesymbol)   
        else:
            hpoid2gene[hpoid] = [genesymbol]

def map2gene(final_matches, CANDIDATE_GENES):
    """ Map to genes for each phenotype without similarity score (one phenotype at one time)

    Args:
        final_matches (list): each element is a tuple (hponame, hpoid, similarity_score)
        CANDIDATE_GENES (list): input candidate genes of the patient
       
    Returns:
        mapped_genes (list): genes associated with the phenotype 

    """
    # Pre-process the final_matches by remove '-synonym' from the hpoid and then unique the hpoids
    final_matches = [_[1] for _ in final_matches] 
    final_matches = [_.split('-')[0] for _ in final_matches] 
    final_matches = list(set(final_matches))
    mapped_genes = []
    for hpoid in final_matches:
        try:
            mapped_genes += hpoid2gene[hpoid] 
        except KeyError:
            pass
    ## For each phenotype in patient record, we generated multiple phenotype keywords 
    ## to maximize the mapping to the genes. However, we only count it once for each 
    ## phenotype even if the multiple keywords lead to multiple mappings.
    ## In practice, we don't need to screen through all genes. Only a few gene candidates are examined.
    mapped_genes = list(set(mapped_genes) & set(CANDIDATE_GENES))
    return mapped_genes

def map2geneWithSim(final_matches, CANDIDATE_GENES):
    """ Map to genes for each phenotype (one phenotype at one time) with similarity score 
    final_matches like: [('Postnatal macrocephaly','HP:0005490',0.5), ('Macrocephaly,relative','HP:0004482-synonym', 0.5), ...]

    Args:
        final_matches (list): each element is a tuple (hponame, hpoid, similarity_score)
        CANDIDATE_GENES (list): input candidate genes of the patient

    Returns:
        mapped_genes_score (dict): a dict with gene symbol as key and similarity score as value 
        mapped_genes_score_phenospecificity (dict): a dict with gene symbol as key and similarity score 
            that is weighted by phenotype rareness as value 

    """
    global hpoidpheno_count
    hpoid_sim = dict() 
    for match in final_matches:
        hpoid = match[1].split('-')[0]
        sim = match[2]
        if hpoid not in hpoid_sim or sim > hpoid_sim[hpoid]:
            hpoid_sim[hpoid] = sim
    mapped_genes_score = dict()
    mapped_genes_score_phenospecificity = dict()

    for hpoid in hpoid_sim:
        try:
            mapped_genes = hpoid2gene[hpoid]
        except KeyError:
            continue
        sim = hpoid_sim[hpoid]
        for gene in mapped_genes:
            # In practice, we don't need to screen through all genes. 
            # Only a few gene candidates are examined.
            if gene not in CANDIDATE_GENES:
                continue
            if gene not in mapped_genes_score or sim > mapped_genes_score[gene]:
                mapped_genes_score[gene] = sim
                if hpoid in hpoidpheno_count:
                    pheno_weight = 6.0 / np.log(hpoidpheno_count[hpoid] + 2.7183) 
                else: 
                    pheno_weight = 6.0  
                mapped_genes_score_phenospecificity[gene] = sim * pheno_weight
    return mapped_genes_score, mapped_genes_score_phenospecificity 

hpoid2disease = dict()
numphenosindisease = dict()
with open(PHENOTYPE_TO_DISEASE_FILE, 'rb') as f:
    # skip the first header line
    f.readline()
    for line in f.readlines():
        line = line.rstrip()
        parts = line.split('\t')
        diseaseid, hpoid, hponame, diseasename = parts[0], parts[1], parts[2], parts[3]
        if diseasename in numphenosindisease:
            numphenosindisease[diseasename] += 1
        else:
            numphenosindisease[diseasename] = 1
        if hpoid in hpoid2disease:
            hpoid2disease[hpoid].append(diseasename)
        else:
            hpoid2disease[hpoid] = [diseasename]

def map2disease(final_matches):
    """ Map to diseases for each phenotype without similarity score (one phenotype at one time)

    Args:
        final_matches (list): each element is a tuple (hponame, hpoid, similarity_score)
       
    Returns:
        mapped_diseases (list): diseases associated with the phenotype 

    """
    # Pre-process the final_matches by remove '-synonym' from the hpoid and then unique the hpoids
    final_matches = [_[1] for _ in final_matches] 
    final_matches = [_.split('-')[0] for _ in final_matches] 
    final_matches = list(set(final_matches))
    mapped_diseases = []
    for hpoid in final_matches:
        try:
            mapped_diseases += hpoid2disease[hpoid] 
        except KeyError:
            pass
    return mapped_diseases

def map2diseaseWithSim(final_matches):
    """ Map to diseases for each phenotype (one phenotype at one time) with similarity score 
    final_matches like: [('Postnatal macrocephaly','HP:0005490',0.5), ('Macrocephaly,relative','HP:0004482-synonym', 0.5), ...]

    Args:
        final_matches (list): each element is a tuple (hponame, hpoid, similarity_score)

    Returns:
        mapped_diseases_score (dict): a dict with diseasename as key and similarity score as value 

    """
    hpoid_sim = dict() 
    for match in final_matches:
        hpoid = match[1].split('-')[0]
        sim = match[2]
        if hpoid not in hpoid_sim or sim > hpoid_sim[hpoid]:
            hpoid_sim[hpoid] = sim
    mapped_diseases_score = dict()
    for hpoid in hpoid_sim:
        try:
            mapped_diseases = hpoid2disease[hpoid]
        except KeyError:
            continue
        sim = hpoid_sim[hpoid]
        for disease in mapped_diseases:
            if disease not in mapped_diseases_score or sim > mapped_diseases_score[disease]:
                mapped_diseases_score[disease] = sim
    return mapped_diseases_score 


def generate_score(phenos, CANDIDATE_GENES, corner_cases, original_phenos):
    """ This is the main function in this .py file. It is called in the main.py file.

    Args:
        phenos (list): input patient's phenotypes
        CANDIDATE_GENES (list): input candidate genes of the patient
        corner_cases (dict): corner cases of input patient's phenotypes that need to be handled. Currently the corner cases are to make growth delay and development delay interchangebly. 
        original_phenos (list): input patient's phenotypes before handling the corner cases

    Returns:
        ranking_genes (list): each element in the list is a tuple (gene, score, hits, phenospecificity_score). 
            It records gene symbol, phenotype hit score for this gene, nubmer of phenotypes associated with this gene, 
            and the phenotype correlation score by weighing the rareness of each phenotype.  
        ranking_diseases (list): each element in the list is a tuple (disease, score, hits). 
            It records the disease name, phenotype hit score for this disease, and number of phenotypes associated with this disease.
        gene_associated_phenos (dict): phenotypes associated with each gene

    """
    all_mapped_genes = []
    all_mapped_genes_score = {}
    gene_phenos = {} 
    all_mapped_diseases = []
    all_mapped_diseases_score = {}
    disease_phenos = {} 

    gene_associated_phenos = dict()
    all_mapped_genes_score_phenospecificity = {}

    for pheno in phenos:
        final_matches = map2hpoWithPhenoSynonyms(pheno)
        mapped_genes = map2gene(final_matches, CANDIDATE_GENES)

        for gene in mapped_genes:
            if pheno not in original_phenos:
                continue
            if gene in gene_associated_phenos:
                gene_associated_phenos[gene].append(pheno)
            else:
                gene_associated_phenos[gene] = [pheno]

        all_mapped_genes += mapped_genes
        mapped_genes_score, mapped_genes_score_phenospecificity = map2geneWithSim(final_matches, CANDIDATE_GENES)
        if pheno in corner_cases:
            pheno = corner_cases[pheno]
        for gene in mapped_genes_score:
            if gene not in all_mapped_genes_score:
                all_mapped_genes_score[gene] = mapped_genes_score[gene]
                all_mapped_genes_score_phenospecificity[gene] = mapped_genes_score_phenospecificity[gene]
            else:
                all_mapped_genes_score[gene] += mapped_genes_score[gene]
                all_mapped_genes_score_phenospecificity[gene] += mapped_genes_score_phenospecificity[gene]
            if gene not in gene_phenos:
                gene_phenos[gene] = [pheno]
            else:
                gene_phenos[gene].append(pheno)

        mapped_diseases = map2disease(final_matches)
        all_mapped_diseases += mapped_diseases
        mapped_diseases_score = map2diseaseWithSim(final_matches)
        for disease in mapped_diseases_score:
            if disease not in all_mapped_diseases_score:
                all_mapped_diseases_score[disease] = mapped_diseases_score[disease]
            else:
                all_mapped_diseases_score[disease] += mapped_diseases_score[disease]
            if disease not in disease_phenos:
                disease_phenos[disease] = [pheno]
            else:
                disease_phenos[disease].append(pheno)

    num_patient_phenos = len(phenos)
    for disease in all_mapped_diseases_score:
        numphenos = numphenosindisease[disease]
        disease_score = all_mapped_diseases_score[disease]
        # algo to calculate the disease score
        all_mapped_diseases_score[disease] = disease_score ** 2 / float(numphenos) / (1 + num_patient_phenos - disease_score)

    all_mapped_genes_score = [(k,v) for k, v in all_mapped_genes_score.items()]   
    all_mapped_genes_score = sorted(all_mapped_genes_score, key = lambda pair: pair[1], reverse = True)
    all_mapped_diseases_score = [(k,v) for k, v in all_mapped_diseases_score.items()]   
    all_mapped_diseases_score = sorted(all_mapped_diseases_score, key = lambda pair: pair[1], reverse = True)

    count = Counter(all_mapped_genes)
    count = sorted(count.items(), key = lambda pair: pair[1], reverse = True)
    count_disease = Counter(all_mapped_diseases)
    count_disease = sorted(count_disease.items(), key = lambda pair: pair[1], reverse = True)

    ranking_genes = []
    ranking_diseases = []
    for id in xrange(len(all_mapped_genes_score)):
        gene, score = all_mapped_genes_score[id]
        hits = None
        phenospecificity_score = all_mapped_genes_score_phenospecificity[gene]
        for gene_hits in count:
            if gene_hits[0] == gene:
                hits = gene_hits[1] 
                break
        if hits:
            ranking_genes.append((gene, score, hits, phenospecificity_score))

    for id in xrange(len(all_mapped_diseases_score)):
        disease, score = all_mapped_diseases_score[id]
        hits = None
        for disease_hits in count_disease:
            if disease_hits[0] == disease:
                hits = disease_hits[1] 
                break
        if hits:
            ranking_diseases.append((disease, score, hits))

    return ranking_genes, ranking_diseases, gene_associated_phenos

In [5]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [6]:
import urllib2
import json

In [7]:
from langdetect import detect

In [8]:
input_phenotype = '甲状腺'

In [10]:
language = detect(unicode(input_phenotype))
if language == "zh-cn" or language == "ko":
    site = "https://translate.googleapis.com/translate_a/single?client=gtx&sl=zh-Hans&tl=en&dt=t&q="+input_phenotype
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive'}
    req = urllib2.Request(site, headers=hdr)
    try:
        page = urllib2.urlopen(req)
        content = page.read()
        input_phenotype = json.loads(content)[0][0][0]
    except urllib2.HTTPError, e:
        print e
        

# match_result = map2hpoWithPhenoSynonyms(input_phenotype)
# match_result = sorted(match_result, key = lambda x: x[2], reverse = True)
# if match_result==[]:
#     match_result = ''
# elif match_result[0][0] == 'Familial  hyperprolactinemia':
#     match_result = ''
# else:
#     if match_result[0][2] == 1.0:
#         match_result = match_result[:1]
#     match_id = [i[1] for i in match_result]
#     for indx,i in enumerate(match_id):
#         if i[-7:]=='synonym':
#             match_id[indx] = match_id[indx][:-8]
#     match_table = chpo[chpo['编号'].isin(match_id)].iloc[:,[3,2,1,0,5]].reset_index(drop=True)

In [12]:
input_phenotype

u'thyroid'

In [249]:
match_result = map2hpoWithPhenoSynonyms(input_phenotype)
match_result = sorted(match_result, key = lambda x: x[2], reverse = True)
if match_result==[]:
    match_result = ''
elif match_result[0][0] == 'Familial  hyperprolactinemia':
    match_result = ''
else:
    if match_result[0][2] == 1.0:
        match_result = match_result[:1]
    match_id = [i[1] for i in match_result]
    for indx,i in enumerate(match_id):
        if i[-7:]=='synonym':
            match_id[indx] = match_id[indx][:-8]
    match_table = chpo[chpo['编号'].isin(match_id)].iloc[:,[3,2,1,0,5]].reset_index(drop=True)
    

In [250]:
match_result

''

In [251]:
match_table

,表型中文名,表型英文名,HPO编号,类别,释义
0,心动过速,Tachycardia,HP:0001649,心血管系统的异常,心脏运动速率极快，通常在100次每秒以上。


In [2]:
import google

In [1]:
from google import google

In [18]:
num_page = 1
search_results = google.search("afraid of light", 1)

In [14]:
search_results[1].name

u'Photophobia - Wikipedia'

In [24]:
list(set([i.name[:-12] for i in search_results if i.name[-9:]=='Wikipedia']))

[u'Photophobia', u'Heliophobia']